In [2]:
import numpy as np
import pandas as pd
import os
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC

from sklearn.preprocessing import MinMaxScaler

#from dfply import arrange

from sklearn.utils import resample
from sklearn.metrics import cohen_kappa_score, make_scorer

In [3]:
#path1 = "D:/tunc_oz/apply_model"
path1 = "/home/rsrg9/Documents/tunc_oz/apply_model"
os.chdir(path1)

# Second path
path2 = "csv_data_r_import/cologne/scale"
os.chdir(path2)

In [141]:
def svm_fit(x, y, index_train=None):
    # Expand coarse grid
    coarse_grid = {'C': 2.0 ** np.arange(-4, 13, 2),
                   'gamma': 2.0 ** np.arange(-5, 4, 2)}
    
    kappa_scorer = make_scorer(cohen_kappa_score)

    # Coarse grid search
    svm_coarse = SVC(kernel='rbf')
    svm_coarse_cv = GridSearchCV(svm_coarse, param_grid=coarse_grid, scoring=kappa_scorer)#, cv=index_train)
    svm_coarse_cv.fit(x, y)
    
    # Get best coarse grid parameters
    best_c = svm_coarse_cv.best_params_['C']
    best_gamma = svm_coarse_cv.best_params_['gamma']
    
    # Define narrow grid borders
    a_gamma = np.log2(best_gamma) - 2
    b_gamma = np.log2(best_gamma) + 2
    a_c = np.log2(best_c) - 2
    b_c = np.log2(best_c) + 2
    
    # Expand narrow grid
    narrow_grid = {'C': 2.0 ** np.arange(a_c, b_c, 0.5),
                   'gamma': 2.0 ** np.arange(a_gamma, b_gamma, 0.5)}
    
    # Narrow grid search
    svm_narrow = SVC(kernel='rbf')
    svm_narrow_cv = GridSearchCV(svm_narrow, param_grid=narrow_grid, scoring=kappa_scorer)#, cv=index_train)
    svm_narrow_cv.fit(x, y)
    
    return svm_narrow_cv

# Usage example:
# svm_model = svm_fit(X_train, y_train, StratifiedKFold(n_splits=10, shuffle=True))

# Euclidean Distance between two points lying in the input space
def euc_dis(a, b):
    temp = 0
    for ii in range(len(a)):
        temp += (a[ii] - b[ii])**2
    return np.sqrt(temp)

# Evaluate the distance between Virtual Support Vectors and Support Vectors lying in the input space
def rem_extrem(org, VSV1, a):
    distance = pd.DataFrame(index=range(len(org)), columns=['label', 'distance'])
    distanceSVC1 = []
    distanceSVC2 = []
    
    for l in range(len(org)):
        distance.loc[l, 'label'] = str(org.iloc[l, -1])
        distance.loc[l, 'distance'] = euc_dis(org.iloc[l, :-1], VSV1.iloc[l, :-1])
    
    SVClass1 = org[org['REF'] == org['REF'].unique()[0]]
    SVClass2 = org[org['REF'] == org['REF'].unique()[1]]
    
    if len(SVClass1) > 0:
        for n in range(len(SVClass1) - 1):
            for nn in range(n, len(SVClass1) - 1):
                distanceSVC1.append(euc_dis(SVClass1.iloc[n, :-1], SVClass1.iloc[n + nn, :-1]))
        disClass1median = np.mean(distanceSVC1)
        boundClass1 = disClass1median * a
    
    if len(SVClass2) > 0:
        for n in range(len(SVClass2) - 1):
            for nn in range(n, len(SVClass2) - 1):
                distanceSVC2.append(euc_dis(SVClass2.iloc[n, :-1], SVClass2.iloc[n + nn, :-1]))
        disClass2median = np.mean(distanceSVC2)
        boundClass2 = disClass2median * a
    
    for k in range(len(org)):
        if np.isnan(distance.loc[k, 'distance']):
            VSV1.iloc[k, :] = np.nan
        else:
            if boundClass1 is not None:
                if distance.loc[k, 'label'] == org['REF'].unique()[0]:
                    if distance.loc[k, 'distance'] > boundClass1:
                        VSV1.iloc[k, :] = np.nan
            else:
                if boundClass2 is not None:
                    if distance.loc[k, 'label'] == org['REF'].unique()[1]:
                        if distance.loc[k, 'distance'] > boundClass2:
                            VSV1.iloc[k, :] = np.nan
    return VSV1

# Kernel distance between two points lying in the hyperspace
def kern_dis(a, b, kernelfunc):
    a = np.array(a).flatten()
    b = np.array(b).flatten()
    dk = np.sqrt(kernelfunc(a, a) + kernelfunc(b, b) - 2 * kernelfunc(a, b))
    return dk

# Evaluate the distance between Virtual Support Vectors and Support Vectors lying in the hyperspace
def rem_extrem_kerneldist(org, VSV1, a, kernelfunc):
    distance = pd.DataFrame(index=range(len(org)), columns=['label', 'distance'])
    distanceSVC1 = []
    distanceSVC2 = []
    
    for l in range(len(org)):
        distance.loc[l, 'label'] = str(org.iloc[l, -1])
        distance.loc[l, 'distance'] = kern_dis(org.iloc[l, :-1], VSV1.iloc[l, :-1], kernelfunc)
    
    SVClass1 = org[org['REF'] == org['REF'].unique()[0]]
    SVClass2 = org[org['REF'] == org['REF'].unique()[1]]
    
    if len(SVClass1) > 0:
        for n in range(len(SVClass1) - 1):
            for nn in range(n, len(SVClass1) - 1):
                distanceSVC1.append(kern_dis(SVClass1.iloc[n, :-1], SVClass1.iloc[n + nn, :-1], kernelfunc))
        disClass1median = np.mean(distanceSVC1)
        boundClass1 = disClass1median * a
    
    if len(SVClass2) > 0:
        for n in range(len(SVClass2) - 1):
            for nn in range(n, len(SVClass2) - 1):
                distanceSVC2.append(kern_dis(SVClass2.iloc[n, :-1], SVClass2.iloc[n + nn, :-1], kernelfunc))
        disClass2median = np.mean(distanceSVC2)
        boundClass2 = disClass2median * a
    
    for k in range(len(org)):
        if np.isnan(distance.loc[k, 'distance']):
            VSV1.iloc[k, :] = np.nan
        else:
            if boundClass1 is not None:
                if distance.loc[k, 'label'] == org['REF'].unique()[0]:
                    if distance.loc[k, 'distance'] > boundClass1:
                        VSV1.iloc[k, :] = np.nan
            else:
                if boundClass2 is not None:
                    if distance.loc[k, 'label'] == org['REF'].unique()[1]:
                        if distance.loc[k, 'distance'] > boundClass2:
                            VSV1.iloc[k, :] = np.nan
    return VSV1

def pred_one(model, data_point):
    # Extract necessary components from the SVM model
    support_vectors = model.n_support_
    kernel_function = model.kernel
    coefficients = model.dual_coef_.ravel()
    intercept = model.intercept_
    
    # Initialize prediction variable
    prediction = 0
    
    # Iterate over each support vector
    for j in range(len(support_vectors)):
        # Compute kernel function value between the j-th support vector and the data point
        kernel_value = kernel_function(data_point.reshape(1, -1), model.support_vectors_[j, :].reshape(1, -1))
        
        # Multiply kernel value by the corresponding coefficient and add to prediction
        weighted_value = kernel_value * coefficients[j]
        prediction += weighted_value
    
    # Subtract intercept to get the final prediction
    final_prediction = prediction - intercept
    
    return final_prediction

def uncertainty_dist_v2_2(org, samp):
    distance = pd.DataFrame(columns=['control_label', 'distance'], index=range(len(samp)))
    
    for k in range(len(samp)):
        distance.loc[k, 'distance'] = np.sign(pred_one(org.finalModel, samp.iloc[k, :-1])) * \
                                      np.where(pred_one(org.finalModel, samp.iloc[k, :-1]) > 0, 1, -1)
    
    # Normalize distance
    preProc = preprocessing.MinMaxScaler()
    preProc.fit(distance[['distance']])
    normdistance = preProc.transform(distance[['distance']])
    
    samp['normdistance'] = normdistance
    
    return samp

def alter_labels(distance_data, ref):
    # Merge features and original labels
    ref_added = pd.concat([distance_data, ref], axis=1)
    # Order by most uncertain samples
    ref_added_or = ref_added.sort_values(by='distance')
    # Re-label most uncertain n number of samples
    ref_added_or.iloc[:250, -1] = ref_added_or.iloc[:250, -2]
    ref_added_or.iloc[:250, -2] = 1.0
    # Re-order dataset by its index
    ref_added_or['index'] = range(len(ref_added_or))
    ref_added_reor = ref_added_or.sort_values(by='index')
    
    # Extract labels for prediction
    labels = ref_added_reor.iloc[:, -5]
    return labels

def ExCsvMSD(datadase, filename=None):
    # Convert to numpy array
    datadase = np.array(datadase)
    n = datadase.shape[1]
    MSDdata = np.empty((2, n), dtype=float)
    
    MSDdata[0, :] = np.mean(datadase, axis=0)
    MSDdata[1, :] = np.std(datadase, axis=0)
    
    MSDdata_final = np.vstack((datadase, MSDdata))
    
    # Export final mean and standard deviation to .csv-file
    if filename is not None:
        pd.DataFrame(MSDdata_final).to_csv(filename, index=False, header=False)
    
    return MSDdata_final

In [152]:
inputPath = "cologne_res_100_L2-L13.csv"
sMax = 1000
bound = [0.3, 0.6, 0.9]
boundMargin = [1.5, 1.0, 0.5]
sampleSizesPor = [40, 25, 16, 12, 10, 8, 6, 4, 3, 2, 1]
colheader = ["40", "25", "16", "12", "10", "8", "6", "4", "3", "2", "1"]
sindexSVMDATA = 36
numFeat = 18
eindexSVMDATA = sindexSVMDATA + numFeat - 1
objInfoNames = ["Lx_g_comp", "Lx_g_elfi", "Lx_g_refi", "Lx_g_roun", "Lx_g_shin",
                "Lx_m_bl", "Lx_m_gr", "Lx_m_ndvi", "Lx_m_nir", "Lx_m_re",
                "Lx_sd_bl", "Lx_sd_gr", "Lx_sd_ndvi", "Lx_sd_nir", "Lx_sd_re",
                "Lx_t_diss", "Lx_t_hom", "Lx_t_mean",
                "label"]
columnClass = [None] * 217 + ["factor", "integer"]

# Import data
preproc_DataPool = pd.read_csv(inputPath, header=0, sep=";", dtype=str, na_values=None)

tmp_DataPool = preproc_DataPool.iloc[:, :-2]

generalDataPool_columns = tmp_DataPool.columns

converters = {col: lambda x: float(x.replace(',', '.')) for col in generalDataPool_columns}
generalDataPool = pd.read_csv(inputPath, header=0, sep=";", na_values=None, converters=converters)


generalDataPool.dropna(subset=["REF"], inplace=True)  # Remove rows with missing REF values
generalDataPool["REF"] = pd.Categorical(generalDataPool["REF"])

# Transform to 2-Class-Case "Bushes Trees" VS rest
first_label_class = generalDataPool["REF"].cat.categories[0]  # Note that the first record is of class "bushes trees"
generalDataPool["REF"] = generalDataPool["REF"].apply(lambda x: first_label_class if x == first_label_class else "other")
generalDataPool["REF"] = pd.Categorical(generalDataPool["REF"])

data = generalDataPool.iloc[:, sindexSVMDATA:eindexSVMDATA + 1]
REF = generalDataPool.iloc[:, -1]
data_with_label = pd.concat([data, REF], axis=1)
data_label = data_with_label.iloc[:, -1]

In [7]:
generalDataPool.dtypes

L02_G_COMP     float64
L02_G_EFIT     float64
L02_G_RFIT     float64
L02_G_ROUN     float64
L02_G_SHIN     float64
                ...   
L13_T_DISS     float64
L13_T_HOM      float64
L13_T_MEA      float64
REF           category
USE              int64
Length: 218, dtype: object

In [18]:
#normalizedFeat['L02_G_COMP'] = normalizedFeat['L02_G_COMP'].replace(',', '.', regex=True)

#normalizedFeat['L02_G_COMP'] = normalizedFeat['L02_G_COMP'].astype(float)

In [ ]:
#normalizedFeat['L02_G_EFIT'] = normalizedFeat['L02_G_EFIT'].apply(lambda x: pd.to_numeric(x.str.replace(',', '.'), errors='coerce'))

In [153]:
normalizedFeat = generalDataPool.iloc[:, :-2]
normalizedLabelUSE = generalDataPool.iloc[:, -2:]

# Scaling
preProc = MinMaxScaler()
normalizedFeatBase = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, sindexSVMDATA:eindexSVMDATA + 1]), columns=objInfoNames[:-1])

# Apply range of basemodel to all levels
normalizedFeat2 = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, :numFeat]), columns=objInfoNames[:-1])
normalizedFeat3 = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, numFeat:(2 * numFeat)]), columns=objInfoNames[:-1])
normalizedFeat5 = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, (3 * numFeat):(4 * numFeat)]), columns=objInfoNames[:-1])
normalizedFeat6 = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, (4 * numFeat):(5 * numFeat)]), columns=objInfoNames[:-1])
normalizedFeat7 = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, (5 * numFeat):(6 * numFeat)]), columns=objInfoNames[:-1])
normalizedFeat8 = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, (6 * numFeat):(7 * numFeat)]), columns=objInfoNames[:-1])
normalizedFeat9 = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, (7 * numFeat):(8 * numFeat)]), columns=objInfoNames[:-1])
normalizedFeat10 = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, (8 * numFeat):(9 * numFeat)]), columns=objInfoNames[:-1])
normalizedFeat11 = pd.DataFrame(preProc.fit_transform(normalizedFeat.iloc[:, (9 * numFeat):(10 * numFeat)]), columns=objInfoNames[:-1])

# Recombine normalized sets to one data frame
normalizedDataPoolAllLev = pd.concat([normalizedFeat2, normalizedFeat3, normalizedFeatBase, normalizedFeat5,
                                      normalizedFeat6, normalizedFeat7, normalizedFeat8, normalizedFeat9,
                                      normalizedFeat10, normalizedFeat11, normalizedLabelUSE], axis=1)

# Remove used temporary variables
del normalizedFeat, normalizedFeat2, normalizedFeat3, normalizedFeatBase, normalizedFeat5, normalizedFeat6
del normalizedFeat7, normalizedFeat8, normalizedFeat9, normalizedFeat10, normalizedFeat11


In [187]:
# Split data into test, train, and validate data
trainDataPoolAllLev, testDataAllLev, validateDataAllLev = [df for _, df in normalizedDataPoolAllLev.groupby('USE')]
trainDataPoolAllLev = trainDataPoolAllLev.iloc[:, :-1]
testDataAllLev = testDataAllLev.iloc[:, :-1]
validateFeatAllLev = validateDataAllLev.iloc[:, :-2]
validateLabels = validateDataAllLev.iloc[:, -2]

# Order train data pool by class label in alphabetical order
trainDataPoolAllLev = trainDataPoolAllLev.sort_values(by=trainDataPoolAllLev.columns[-1])

# Current training data-set, updated (refreshed) after each iteration
trainDataCur = trainDataPoolAllLev.copy()
testDataCur = testDataAllLev.copy()

# Set randomized seed for the random sampling procedure
seed = 5

# Initial seed value for randomized sampling
seed += np.random.randint(1, 101)

# Definition of apriori-probabilities
pA = pB = pC = pD = pE = pF = 1 / 6

# Definition of training sample set sizes S [% of max. sample size]
sCur = sMax * (sampleSizesPor[0] / 100)
# Definition of sample shares
nA, nB, nC, nD, nE, nF = [round(sCur * p) for p in [pA, pB, pC, pD, pE, pF]]
shares = np.array([nA, nB, nC, nD, nE, nF])

# Set randomized seed for the random sampling procedure
np.random.seed(seed)

In [116]:
validateDataAllLev.columns[-2]

'REF'

In [32]:
#stratSamp = trainDataCur.groupby('REF', observed=False)[trainDataCur.columns].apply(lambda x: x.sample(67, replace=False))

In [188]:
# Define the sampling function
def sample_within_group(group):
    # Add the original IDs as a new column
    group['ID_unit'] = group.index
    
    # Perform sampling within the group
    sampled_group = group.sample(min(len(group), 67), replace=False)
    
    return sampled_group

# Apply the sampling function to each group
stratSamp = trainDataCur.groupby('REF', observed=False).apply(sample_within_group)

# Reset the index to obtain a flat DataFrame with the original IDs preserved
stratSamp.reset_index(drop=True, inplace=True)

# Get samples of trainDataCur and set trainDataCur new
trainDataCurRemaining = trainDataCur.drop(stratSamp["ID_unit"])

# Split test feat from test label for later join with trainData
trainFeat = stratSamp.iloc[:, :len(trainDataPoolAllLev.columns)-1]
trainLabels = stratSamp.iloc[:, len(trainDataPoolAllLev.columns)-1]

/tmp/ipykernel_42210/1510426551.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stratSamp = trainDataCur.groupby('REF', observed=False).apply(sample_within_group)


In [101]:
len(trainDataPoolAllLev.columns)

181

In [ ]:
trainFeat.columns

In [104]:
stratSamp.columns[len(trainDataPoolAllLev.columns)-1]

'REF'

In [30]:
#stratSamp = trainDataCur.groupby('REF',observed=False)
#stratSamp = stratSamp.apply(lambda x: x.sample(67, replace=False))

In [ ]:
# Definition of sampling configuration (strata: random sampling without replacement)
#stratSamp = trainDataCur.groupby('REF').apply(lambda x: x.sample(shares, replace=False)).reset_index(drop=True)

In [46]:
trainDataCur.columns[-1]

'Lx_t_diss'

In [ ]:
stratSamp.iloc[:,181]

In [ ]:
stratSamp["ID_unit"]

In [ ]:
trainFeat.describe

In [189]:

# Subset for each outer iteration test data to speed up computing
testDataCur = testDataCur.sort_values(by=testDataCur.columns[-1])
# Apply the sampling function to each group
stratSamp = testDataCur.groupby('REF', observed=False).apply(sample_within_group)

# Split test feat from test label for later join with trainData
testFeat = stratSamp.iloc[:, :len(testDataCur.columns)-1]
testLabels = stratSamp.iloc[:, len(testDataCur.columns)-1]

# Subset on base level
testFeatsub = testFeat.iloc[:, sindexSVMDATA:eindexSVMDATA + 1]

# TrainData index to split between train and test in svmFit
countTrainData = trainFeat.shape[0]
indexTrainData = [list(range(1, countTrainData + 1))]

# SVM base for invariants

# Subset on L_4
trainFeat = trainFeat.iloc[:, sindexSVMDATA:eindexSVMDATA + 1]

# Join train and test data (separable through indexTrainData in svmFit)
tuneFeat = pd.concat([trainFeat, testFeatsub], axis=0)
tuneLabel = np.concatenate((trainLabels.values, testLabels.values))

validateFeatsub = validateFeatAllLev.iloc[:, sindexSVMDATA:eindexSVMDATA+1]

/tmp/ipykernel_42210/3587032453.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stratSamp = testDataCur.groupby('REF', observed=False).apply(sample_within_group)


In [76]:
from sklearn import datasets
iris = datasets.load_iris()
# Expand coarse grid
coarse_grid = {'C': 2.0 ** np.arange(-4, 13, 2),
                'gamma': 2.0 ** np.arange(-5, 4, 2)}


# Coarse grid search
svm_coarse = SVC(kernel='rbf')
svm_coarse_cv = GridSearchCV(svm_coarse, param_grid=coarse_grid, scoring='accuracy')#, cv=indexTrainData)
svm_coarse_cv.fit(iris.data, iris.target)

GridSearchCV(estimator=SVC(),
             param_grid={'C': array([6.250e-02, 2.500e-01, 1.000e+00, 4.000e+00, 1.600e+01, 6.400e+01,
       2.560e+02, 1.024e+03, 4.096e+03]),
                         'gamma': array([0.03125, 0.125  , 0.5    , 2.     , 8.     ])},
             scoring='accuracy')

In [ ]:
tuneFeat.describe

In [ ]:
tuneLabel

In [109]:
# Expand coarse grid
coarse_grid = {'C': 2.0 ** np.arange(-4, 13, 2),
                'gamma': 2.0 ** np.arange(-5, 4, 2)}

# Coarse grid search
svm_coarse = SVC(kernel='rbf')
svm_coarse_cv = GridSearchCV(svm_coarse, param_grid=coarse_grid, scoring='accuracy')#, cv=indexTrainData)
svm_coarse_cv.fit(tuneFeat, tuneLabel)

GridSearchCV(estimator=SVC(),
             param_grid={'C': array([6.250e-02, 2.500e-01, 1.000e+00, 4.000e+00, 1.600e+01, 6.400e+01,
       2.560e+02, 1.024e+03, 4.096e+03]),
                         'gamma': array([0.03125, 0.125  , 0.5    , 2.     , 8.     ])},
             scoring='accuracy')

In [110]:
# Get best coarse grid parameters
best_c = svm_coarse_cv.best_params_['C']
best_gamma = svm_coarse_cv.best_params_['gamma']

# Define narrow grid borders
a_gamma = np.log2(best_gamma) - 2
b_gamma = np.log2(best_gamma) + 2
a_c = np.log2(best_c) - 2
b_c = np.log2(best_c) + 2

# Expand narrow grid
narrow_grid = {'C': 2.0 ** np.arange(a_c, b_c, 0.5),
                'gamma': 2.0 ** np.arange(a_gamma, b_gamma, 0.5)}

# Narrow grid search
svm_narrow = SVC(kernel='rbf')
svm_narrow_cv = GridSearchCV(svm_narrow, param_grid=narrow_grid, scoring='accuracy')#, cv=indexTrainData)
svm_narrow_cv.fit(tuneFeat, tuneLabel)


GridSearchCV(estimator=SVC(),
             param_grid={'C': array([ 4.        ,  5.65685425,  8.        , 11.3137085 , 16.        ,
       22.627417  , 32.        , 45.254834  ]),
                         'gamma': array([0.125     , 0.1767767 , 0.25      , 0.35355339, 0.5       ,
       0.70710678, 1.        , 1.41421356])},
             scoring='accuracy')

In [194]:
# SVM parameter tuning
svm_model=svm_fit(trainFeat, trainLabels, indexTrainData)

In [ ]:
validateLabels

In [195]:

# Predict labels of test data
predLabelsSVM = svm_model.predict(validateFeatsub)

# Accuracy assessment
accSVM = accuracy_score(validateLabels, predLabelsSVM)
print(accSVM)

0.8637477761810616


In [198]:
# VSVM on all Level SV
SVindex = svm_model.best_estimator_.support_  # Indices of support vectors
SVtotal = trainDataCur.iloc[SVindex, sindexSVMDATA:eindexSVMDATA+1].reset_index(drop=True)  # Get support vectors

SVL2 = trainDataCur.iloc[SVindex, sindexSVMDATA - 2 * numFeat:sindexSVMDATA - numFeat ].reset_index(drop=True)
SVL3 = trainDataCur.iloc[SVindex, sindexSVMDATA - numFeat:sindexSVMDATA ].reset_index(drop=True)

SVL5 = trainDataCur.iloc[SVindex, sindexSVMDATA + numFeat:sindexSVMDATA + 2 * numFeat ].reset_index(drop=True)
SVL6 = trainDataCur.iloc[SVindex, sindexSVMDATA + 2 * numFeat:sindexSVMDATA + 3 * numFeat ].reset_index(drop=True)
SVL7 = trainDataCur.iloc[SVindex, sindexSVMDATA + 3 * numFeat:sindexSVMDATA + 4 * numFeat ].reset_index(drop=True)
SVL8 = trainDataCur.iloc[SVindex, sindexSVMDATA + 4 * numFeat:sindexSVMDATA + 5 * numFeat ].reset_index(drop=True)
SVL9 = trainDataCur.iloc[SVindex, sindexSVMDATA + 5 * numFeat:sindexSVMDATA + 6 * numFeat ].reset_index(drop=True)
SVL10 = trainDataCur.iloc[SVindex, sindexSVMDATA + 6 * numFeat:sindexSVMDATA + 7 * numFeat ].reset_index(drop=True)
SVL11 = trainDataCur.iloc[SVindex, sindexSVMDATA + 7 * numFeat:sindexSVMDATA + 8 * numFeat ].reset_index(drop=True)


# Bind original SV with modified to new train data set
SVinvar = pd.concat([SVtotal, SVL2, SVL3, SVL5, SVL6, SVL7, SVL8, SVL9, SVL10, SVL11],ignore_index=True)

# Split for training to feature and label
trainFeatVSVM = SVinvar.iloc[:, :-1].reset_index(drop=True)
trainLabelsVSVM = SVinvar.iloc[:, -1]

# Get list with index of train data to split between train and test in svmFit
countTrainData = SVinvar.shape[0]
indexTrainData = [list(range(1, countTrainData + 1))]

In [ ]:
# Join of train and test test data (through indexTrainData in svmFit separable)
tuneFeatVSVM = pd.concat([trainFeatVSVM, testFeatsub])
tuneLabelsVSVM = np.concatenate((trainLabelsVSVM.values, testLabels.values))

# VSVM parameter tuning
tunedVSVM = svm_fit(tuneFeatVSVM, tuneLabelsVSVM)

# Run classification and accuracy assessment for modified SV
predLabelsVSVM = tunedVSVM.predict(validateFeatsub)
accVSVM = accuracy_score(validateLabels, predLabelsVSVM)
print(accVSVM)

In [ ]:
pd.merge(trainFeatVSVM, testFeatsub, how='inner')

In [204]:
trainFeatVSVM.columns

Index(['Lx_g_comp', 'Lx_g_elfi', 'Lx_g_refi', 'Lx_g_roun', 'Lx_g_shin',
       'Lx_m_bl', 'Lx_m_gr', 'Lx_m_ndvi', 'Lx_m_nir', 'Lx_m_re', 'Lx_sd_bl',
       'Lx_sd_gr', 'Lx_sd_ndvi', 'Lx_sd_nir', 'Lx_sd_re', 'Lx_t_diss',
       'Lx_t_hom'],
      dtype='object')

In [205]:
testFeatsub.columns

Index(['Lx_g_comp', 'Lx_g_elfi', 'Lx_g_refi', 'Lx_g_roun', 'Lx_g_shin',
       'Lx_m_bl', 'Lx_m_gr', 'Lx_m_ndvi', 'Lx_m_nir', 'Lx_m_re', 'Lx_sd_bl',
       'Lx_sd_gr', 'Lx_sd_ndvi', 'Lx_sd_nir', 'Lx_sd_re', 'Lx_t_diss',
       'Lx_t_hom', 'Lx_t_mean'],
      dtype='object')

In [200]:
trainFeatVSVM[0:10].reset_index(drop=True)

,Lx_g_comp,Lx_g_elfi,Lx_g_refi,Lx_g_roun,Lx_g_shin,Lx_m_bl,Lx_m_gr,Lx_m_ndvi,Lx_m_nir,Lx_m_re,Lx_sd_bl,Lx_sd_gr,Lx_sd_ndvi,Lx_sd_nir,Lx_sd_re,Lx_t_diss,Lx_t_hom
0,0.095265,0.529954,0.755812,0.216550,0.217151,0.043132,0.042229,0.524556,0.084285,0.045613,0.061682,0.063318,0.380959,0.161381,0.061692,0.142263,0.033707
1,0.068239,0.805668,0.873817,0.141781,0.092888,0.033785,0.032189,0.620288,0.100116,0.036525,0.040698,0.037860,0.405775,0.165309,0.044026,0.090419,0.052963
2,0.068239,0.805668,0.873817,0.141781,0.092888,0.033785,0.032189,0.620288,0.100116,0.036525,0.040698,0.037860,0.405775,0.165309,0.044026,0.090419,0.052963
3,0.073022,0.670330,0.798266,0.201985,0.077653,0.021294,0.014574,0.631935,0.061670,0.007359,0.026509,0.022434,0.316412,0.119451,0.020073,0.098026,0.060738
4,0.073022,0.670330,0.798266,0.201985,0.077653,0.021294,0.014574,0.631935,0.061670,0.007359,0.026509,0.022434,0.316412,0.119451,0.020073,0.098026,0.060738
5,0.050208,0.831325,0.858572,0.097416,0.048185,0.028829,0.029048,0.778228,0.137015,0.022815,0.028229,0.023671,0.250152,0.130107,0.030528,0.124826,0.042560
6,0.050208,0.831325,0.858572,0.097416,0.048185,0.028829,0.029048,0.778228,0.137015,0.022815,0.028229,0.023671,0.250152,0.130107,0.030528,0.124826,0.042560
7,0.073022,0.670330,0.798266,0.201985,0.077653,0.021294,0.014574,0.631935,0.061670,0.007359,0.026509,0.022434,0.316412,0.119451,0.020073,0.098026,0.060738
8,0.094363,0.594595,0.789575,0.273995,0.186283,0.029562,0.038956,0.658483,0.126350,0.045296,0.087441,0.108569,0.624496,0.336826,0.159947,0.146974,0.050292
9,0.094363,0.594595,0.789575,0.273995,0.186283,0.029562,0.038956,0.658483,0.126350,0.045296,0.087441,0.108569,0.624496,0.336826,0.159947,0.146974,0.050292


In [201]:
testFeatsub[0:10].reset_index(drop=True)

,Lx_g_comp,Lx_g_elfi,Lx_g_refi,Lx_g_roun,Lx_g_shin,Lx_m_bl,Lx_m_gr,Lx_m_ndvi,Lx_m_nir,Lx_m_re,Lx_sd_bl,Lx_sd_gr,Lx_sd_ndvi,Lx_sd_nir,Lx_sd_re,Lx_t_diss,Lx_t_hom,Lx_t_mean
0,0.170638,0.175141,0.547923,0.373950,0.335203,0.035360,0.029126,0.633638,0.088880,0.025840,0.054977,0.048150,0.384791,0.177123,0.037086,0.140759,0.033392,0.779395
1,0.053001,0.720588,0.857845,0.143951,0.106342,0.028123,0.020053,0.479045,0.046451,0.019651,0.036058,0.027941,0.613459,0.134085,0.032637,0.106267,0.058016,0.800984
2,0.049515,0.890909,0.887633,0.016153,0.049614,0.040455,0.037858,0.665125,0.118128,0.038382,0.125921,0.114288,0.801519,0.394055,0.118582,0.122787,0.046085,0.793211
3,0.134584,0.642276,0.793205,0.260702,0.123789,0.032374,0.026021,0.546970,0.069121,0.027834,0.053448,0.043370,0.399808,0.149164,0.052509,0.146873,0.035488,0.788992
4,0.037543,0.714286,0.890626,0.063433,0.036538,0.040562,0.038807,0.799970,0.170582,0.033094,0.110431,0.083997,0.422837,0.317054,0.072970,0.167546,0.015673,0.775228
5,0.057376,0.735849,0.830359,0.110736,0.054884,0.046727,0.043753,0.700948,0.145417,0.046122,0.064467,0.053819,0.383194,0.237479,0.069540,0.157800,0.032199,0.780111
6,0.128324,0.538462,0.759943,0.245565,0.156693,0.051341,0.061375,0.514011,0.104413,0.066220,0.155476,0.138538,0.449892,0.146757,0.136889,0.262844,0.024569,0.819940
7,0.057376,0.735849,0.830359,0.110736,0.054884,0.046727,0.043753,0.700948,0.145417,0.046122,0.064467,0.053819,0.383194,0.237479,0.069540,0.157800,0.032199,0.780111
8,0.110442,0.448889,0.694736,0.318567,0.263296,0.018281,0.013568,0.568207,0.057481,0.014487,0.055461,0.054716,0.474513,0.154968,0.068994,0.145481,0.044087,0.826634
9,0.063021,0.684211,0.825306,0.150798,0.159881,0.040094,0.037353,0.554337,0.084519,0.039136,0.051960,0.049767,0.458626,0.210503,0.053680,0.100770,0.058101,0.788544


In [179]:
pd.concat([SVtotal,SVL2,SVL5])

,Lx_g_comp,Lx_g_elfi,Lx_g_refi,Lx_g_roun,Lx_g_shin,Lx_m_bl,Lx_m_gr,Lx_m_ndvi,Lx_m_nir,Lx_m_re,Lx_sd_bl,Lx_sd_gr,Lx_sd_ndvi,Lx_sd_nir,Lx_sd_re,Lx_t_diss,Lx_t_hom,Lx_t_mean
0,0.050208,0.831325,0.858572,0.097416,0.048185,0.028829,0.029048,0.778228,0.137015,0.022815,0.028229,0.023671,0.250152,0.130107,0.030528,0.124826,0.042560,0.785337
1,0.050208,0.831325,0.858572,0.097416,0.048185,0.028829,0.029048,0.778228,0.137015,0.022815,0.028229,0.023671,0.250152,0.130107,0.030528,0.124826,0.042560,0.785337
2,0.094363,0.594595,0.789575,0.273995,0.186283,0.029562,0.038956,0.658483,0.126350,0.045296,0.087441,0.108569,0.624496,0.336826,0.159947,0.146974,0.050292,0.826420
3,0.094363,0.594595,0.789575,0.273995,0.186283,0.029562,0.038956,0.658483,0.126350,0.045296,0.087441,0.108569,0.624496,0.336826,0.159947,0.146974,0.050292,0.826420
4,0.073022,0.670330,0.798266,0.201985,0.077653,0.021294,0.014574,0.631935,0.061670,0.007359,0.026509,0.022434,0.316412,0.119451,0.020073,0.098026,0.060738,0.803718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,0.128719,0.450644,0.696731,0.415129,0.209054,0.033694,0.046634,0.732685,0.160815,0.046254,0.042091,0.036228,0.225737,0.110220,0.048571,0.192499,0.038910,0.788608
85,0.048208,0.821429,0.893459,0.113375,0.126157,0.033787,0.043146,0.646950,0.127958,0.049292,0.047404,0.037481,0.187865,0.130733,0.036498,0.197834,0.042960,0.807497
86,0.048208,0.821429,0.893459,0.113375,0.126157,0.033787,0.043146,0.646950,0.127958,0.049292,0.047404,0.037481,0.187865,0.130733,0.036498,0.197834,0.042960,0.807497
87,0.048208,0.821429,0.893459,0.113375,0.126157,0.033787,0.043146,0.646950,0.127958,0.049292,0.047404,0.037481,0.187865,0.130733,0.036498,0.197834,0.042960,0.807497


In [ ]:


# VSVM - EVALUATION of all Level VSV
actKappa = 0
bestFittingModel = None

# Iteration over bound to test different bound thresholds determining the radius of acceptance
for jj, bound_val in enumerate(bound, start=1):
    SVinvarRadi_list = []
    
    # Iterating over boundMargin to test different thresholds on margin distance
    for kk, bound_margin_val in enumerate(boundMargin, start=1):
        SVinvar_list = []
        
        # Iterate over SVinvarRadi and evaluate distance to hyperplane
        for m in range(len(SVinvarRadi)):
            signa = pred_one(tunedSVM.finalModel, SVinvarRadi[m, :-1])
            if SVinvarRadi[m, -1] == levels(generalDataPool.REF)[0]:
                if -bound_margin_val < signa < bound_margin_val:
                    SVinvar_list.append(SVinvarRadi[m, :])
            else:
                if -bound_margin_val < signa < bound_margin_val:
                    SVinvar_list.append(SVinvarRadi[m, :])

        SVinvar = pd.DataFrame(SVinvar_list, columns=objInfoNames)
        
        # Merge elected VSV with original SV
        SVinvar_org = pd.concat([SVtotal, SVinvar])

        # Split for training to feature and label
        trainFeatVSVM = SVinvar_org.iloc[:, :-1]
        trainLabelsVSVM = SVinvar_org.iloc[:, -1]

        # Get list with index of trainData to split between train and test in svmFit
        countTrainData = SVinvar_org.shape[0]
        indexTrainData = [list(range(1, countTrainData + 1))]

        # Join of train and test data (through indexTrainData in svmFit separable)
        names = objInfoNames[:-1]
        tuneFeatVSVM = pd.concat([trainFeatVSVM, testFeatsub], axis=0)
        tuneFeatVSVM.columns = names
        tuneLabelsVSVM = np.concatenate((trainLabelsVSVM.values, testLabels.values))

        ######################################## VSVM control parameter tuning ########################################
        tunedVSVM = SVC(kernel='linear')
        tunedVSVM.fit(tuneFeatVSVM, tuneLabelsVSVM)

        # Get the best fitting model based on Kappa
        if actKappa < tunedVSVM.resample.Kappa:
            bestFittingModel = tunedVSVM
            actKappa = tunedVSVM.resample.Kappa

# Run classification and accuracy assessment for the best bound setting
# Predict labels of test data
predLabelsVSVMsum = bestFittingModel.predict(validateFeatsub)

# Accuracy assessment
accVSVM_SL = accuracy_score(validateLabels, predLabelsVSVMsum)


In [ ]:



# Balanced & Random unlabeled samples
# Balanced samples

# Definition of sampling configuration (strata: random sampling without replacement)
stratSampRemaining_b = resample(trainDataCurRemaining, n_samples=[b, b, b, b, b, b], replace=False)
samplesRemaining_b = trainDataCurRemaining.iloc[stratSampRemaining_b]

trainDataCurRemaining_b = samplesRemaining_b.iloc[:, :-1]
trainDataCurRemainingsub_b = trainDataCurRemaining_b.iloc[:, sindexSVMDATA:eindexSVMDATA]
REF_b = bestFittingModel.predict(trainDataCurRemainingsub_b)

SVindexUn_b = np.arange(1, len(trainDataCurRemainingsub_b) + 1)
SVtotalUn_b = trainDataCurRemaining_b.iloc[SVindexUn_b - 1, sindexSVMDATA:eindexSVMDATA]
SVtotalUn_b['REF'] = REF_b

SVL2Un_b = trainDataCurRemaining.iloc[SVindexUn_b - 1, sindexSVMDATA - 2*numFeat:sindexSVMDATA - numFeat - 1].copy()
SVL2Un_b['REF'] = REF_b
SVL3Un_b = trainDataCurRemaining.iloc[SVindexUn_b - 1, sindexSVMDATA - numFeat:sindexSVMDATA - 1].copy()
SVL3Un_b['REF'] = REF_b
SVL5Un_b = trainDataCurRemaining_b.iloc[SVindexUn_b - 1, sindexSVMDATA + numFeat:sindexSVMDATA + 2*numFeat - 1].copy()
SVL5Un_b['REF'] = REF_b
SVL6Un_b = trainDataCurRemaining_b.iloc[SVindexUn_b - 1, sindexSVMDATA + 2*numFeat:sindexSVMDATA + 3*numFeat - 1].copy()
SVL6Un_b['REF'] = REF_b
SVL7Un_b = trainDataCurRemaining_b.iloc[SVindexUn_b - 1, sindexSVMDATA + 3*numFeat:sindexSVMDATA + 4*numFeat - 1].copy()
SVL7Un_b['REF'] = REF_b
SVL8Un_b = trainDataCurRemaining_b.iloc[SVindexUn_b - 1, sindexSVMDATA + 4*numFeat:sindexSVMDATA + 5*numFeat - 1].copy()
SVL8Un_b['REF'] = REF_b
SVL9Un_b = trainDataCurRemaining_b.iloc[SVindexUn_b - 1, sindexSVMDATA + 5*numFeat:sindexSVMDATA + 6*numFeat - 1].copy()
SVL9Un_b['REF'] = REF_b
SVL10Un_b = trainDataCurRemaining_b.iloc[SVindexUn_b - 1, sindexSVMDATA + 6*numFeat:sindexSVMDATA + 7*numFeat - 1].copy()
SVL10Un_b['REF'] = REF_b
SVL11Un_b = trainDataCurRemaining_b.iloc[SVindexUn_b - 1, sindexSVMDATA + 7*numFeat:sindexSVMDATA + 8*numFeat - 1].copy()
SVL11Un_b['REF'] = REF_b

SVinvarUn_b = pd.concat([SVtotalUn_b, SVL2Un_b, SVL3Un_b, SVL5Un_b, SVL6Un_b, SVL7Un_b, SVL8Un_b, SVL9Un_b, SVL10Un_b, SVL11Un_b])

# Balanced Unlabeled samples

actKappa = 0

for jj in range(len(bound)):
    SVinvarRadiUn_b_list = []
    for m in range(len(SVinvarRadiUn_b)):
        signa = pred_one(tunedSVM.finalModel, SVinvarRadiUn_b.iloc[m, :-1])
        if SVinvarRadiUn_b.iloc[m, -1] == levels(generalDataPool.REF)[0]:
            if -bound_margin_val < signa < bound_margin_val:
                SVinvarRadiUn_b_list.append(SVinvarRadiUn_b.iloc[m, :])
        else:
            if -bound_margin_val < signa < bound_margin_val:
                SVinvarRadiUn_b_list.append(SVinvarRadiUn_b.iloc[m, :])

    SVinvarUn_b = pd.DataFrame(SVinvarRadiUn_b_list, columns=objInfoNames)
    
    SVinvar_orgUn_b = pd.concat([SVtotal, SVinvarUn_b])

    trainFeatVSVMUn_b = SVinvar_orgUn_b.iloc[:, :-1]
    trainLabelsVSVMUn_b = SVinvar_orgUn_b.iloc[:, -1]

    countTrainDataUn_b = SVinvar_orgUn_b.shape[0]
    indexTrainDataUn_b = [list(range(1, countTrainDataUn_b + 1))]

    names = objInfoNames[:-1]
    tuneFeatVSVMUn_b = pd.concat([trainFeatVSVMUn_b, testFeatsub], axis=0)
    tuneFeatVSVMUn_b.columns = names
    tuneLabelsVSVMUn_b = np.concatenate((trainLabelsVSVMUn_b.values, testLabels.values))

    tunedVSVMUn_b = SVC(kernel='linear')
    tunedVSVMUn_b.fit(tuneFeatVSVMUn_b, tuneLabelsVSVMUn_b)

    if actKappa < tunedVSVMUn_b.resample.Kappa:
        bestFittingModelUn_b = tunedVSVMUn_b
        actKappa = tunedVSVMUn_b.resample.Kappa

# Run classification and accuracy assessment for the best bound setting
predLabelsVSVMsumUn_b = bestFittingModelUn_b.predict(validateFeatsub)

# Accuracy assessment
accVSVM_SL_Un_b = accuracy_score(validateLabels, predLabelsVSVMsumUn_b)


In [ ]:
# Add predicted labels to the features data set
predLabelsVSVMsumUn_unc = pd.concat([validateFeatsub, pd.DataFrame(predLabelsVSVMsumUn_b, columns=["Predicted_Labels"])], axis=1)
predLabelsVSVMsumUn_unc.columns = objInfoNames

# Calculate uncertainty of the samples by selecting SV's and data set
normdistvsvm_sl_un = uncertainty_dist_v2_2(bestFittingModelUn_b, predLabelsVSVMsumUn_unc)

# Alter labels
predlabels_vsvm_Slu = alter_labels(normdistvsvm_sl_un, validateLabels)

# Accuracy assessment
accVSVM_SL_Un_b_ad = accuracy_score(validateLabels, predlabels_vsvm_Slu)
